# Assignment 3.3

# Image Caption Retrieval Model

### 1. Data preprocessing

We will use Microsoft COCO (Common Objects in Context) data set to train our "Image Caption Retrieval Model". This data set consists of pretrained 10-crop VGG19 features (Neural codes) and its corresponding text caption. 


In [5]:
from __future__ import print_function

import os
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict

DATA_PATH = '../data'
EMBEDDING_PATH = '../embeddings'
MODEL_PATH = '../models'

You will need to create above directories and locate data set provided in directory 'data'

#### Reading pairs of image (VGG19 features) - caption data

In [6]:
# DO NOT CHANGE BELOW CODE

import collections

np_train_data = np.load(os.path.join(DATA_PATH,'train_data.npy'))
np_val_data = np.load(os.path.join(DATA_PATH,'val_data.npy'))

train_data = collections.OrderedDict()
for i in range(len(np_train_data.item())):
    cap =  np_train_data.item()['caps']
    img =  np_train_data.item()['ims']
    train_data['caps'] = cap
    train_data['ims'] = img
    
val_data = collections.OrderedDict()
for i in range(len(np_val_data.item())):
    cap =  np_val_data.item()['caps']
    img =  np_val_data.item()['ims']
    val_data['caps'] = cap
    val_data['ims'] = img

In [88]:
# example of caption
train_data['caps'][23]

b'a woman is working in a kitchen carrying a soft toy'

In [46]:
# example of pre-computed VGG19 features
val_data['ims'][1]

array([0.02239205, 0.00604904, 0.02322354, ..., 0.        , 0.00106503,
       0.00711824], dtype=float32)

#### Reading caption and information about its corresponding raw images from Microsoft COCO website

In [13]:
# DO NOT CHANGE BELOW CODE
# use them for your own additional preprocessing step
# to map precomputed features and location of raw images 

import json

with open(os.path.join(DATA_PATH,'instances_val2014.json')) as json_file:
    coco_instances_val = json.load(json_file)
    
with open(os.path.join(DATA_PATH,'captions_val2014.json')) as json_file:
    coco_caption_val = json.load(json_file)

#### Additional preprocessing

In [43]:
# create your own function to map pairs of precomputed features and filepath of raw images
# this will be used later for visualization part
# simple approach: based on matched text caption (see json file)

# YOUR CODE HERE 
def return_imagepair(number):
    img = coco_instances_val['images'][number]['flickr_url']
    features = val_data['ims'][number]
    return(img, features)

print(return_imagepair(4999))

('http://farm8.staticflickr.com/7396/8750681361_391310447e_z.jpg', array([0.        , 0.        , 0.00112501, ..., 0.        , 0.        ,
       0.        ], dtype=float32))


#### Build vocabulary index 

In [44]:
# DO NOT CHANGE BELOW CODE

def build_dictionary(text):

    wordcount = OrderedDict()
    for cc in text:
        words = cc.split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 0
            wordcount[w] += 1
    words = list(wordcount.keys())
    freqs = list(wordcount.values())
    sorted_idx = np.argsort(freqs)[::-1]
    

    worddict = OrderedDict()
    worddict['<pad>'] = 0
    worddict['<unk>'] = 1
    for idx, sidx in enumerate(sorted_idx):
        worddict[words[sidx]] = idx+2  # 0: <pad>, 1: <unk>
    

    return worddict

# use the resulting vocabulary index as your look up dictionary
# to transform raw text into integer sequences

all_captions = []
all_captions = train_data['caps'] + val_data['caps']

# decode bytes to string format
caps = []
for w in all_captions:
    caps.append(w.decode())
    
words_indices = build_dictionary(caps)
print ('Dictionary size: ' + str(len(words_indices)))
indices_words = dict((v,k) for (k,v) in words_indices.items())

Dictionary size: 11473


### 2. Image - Caption Retrieval Model

### Image model

In [97]:
from keras.layers import Input, Dense
from keras.models import Model

inputs = Input(shape=(4096,))
image_dense = Dense(1024, activation='relu')(inputs)


### Caption model

In [53]:
# For embedding layer, initialize with pretrained word embedding (GloVe)

# Set up the glove embedding
GLOVE_DIR = '../glove.6B/'

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [91]:
# Transform all captions into integer sequences for the NN
#words_indices['rowboat']
#embeddings_index['rowboat']
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(caps)
sequences = tokenizer.texts_to_sequences(caps)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1

In [96]:
from keras.preprocessing.sequence import pad_sequences

padded_caps = pad_sequences(sequences, maxlen=15)

In [95]:
# Create the embedding matrix
from numpy import zeros

embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [114]:
from keras.layers import Flatten, Input, Dense, Embedding, Reshape, GRU, merge 
from keras.layers import LSTM, Dropout, BatchNormalization, Activation, TimeDistributed, Dot
from keras.models import Sequential

# Create the caption model
inputs2 = Input(shape=(15,))
embed = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputs2)
lstm = LSTM(256, return_sequences=True)(embed)
dense = TimeDistributed(Dense(1024, activation='relu'))(lstm)
caption = Flatten()(dense)

AttributeError: 'Tensor' object has no attribute 'summary'

### Join model

In [113]:
# YOUR CODE HERE
product = merge([image_dense, caption_dense], mode='dot', dot_axes=(1,1))
# layer for computing dot product between tensors

/Users/daphne/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/Users/daphne/anaconda/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


ValueError: Dimension incompatibility using dot mode: 1024 != 15. Layer shapes: (None, 1024), (None, 15, 1024)

### Main model for training stage

In [ ]:
# YOUR CODE HERE

# define your model input and output

print ("loading the training model")
training_model = Model(inputs=, outputs=)

### Retrieval model

In [ ]:
# YOUR CODE HERE

# define your model input and output

print ("loading sub-models for retrieving Neural codes")
caption_model = Model(inputs=, outputs=)
image_model = Model(inputs=, outputs=)

### Loss function

We define our loss function as a loss for maximizing the margin between a positive and
negative example.  If we call $p_i$ the score of the positive pair of the $i$-th example, and $n_i$ the score of the negative pair of that example, the loss is:

\begin{equation*}
loss = \sum_i{max(0, 1 -p_i + n_i)}
\end{equation*}

In [ ]:
from keras import backend as K


def max_margin_loss(y_true, y_pred):
    
    # YOUR CODE HERE
    loss_ =
    
    return loss_
   

#### Accuracy metric for max-margin loss
How many times did the positive pair effectively get a higher value than the negative pair?

In [ ]:
# YOUR CODE HERE
def accuracy(y_true, y_pred):
    
    # YOUR CODE HERE
    accuracy_ =
    
    return accuracy_


### Compile model

In [ ]:
# DO NOT CHANGE BELOW CODE
print ("compiling the training model")
training_model.compile(optimizer='adam', loss=max_margin_loss, metrics=[accuracy])

### 3. Data preparation for training the model

* adjust the length of captions into fixed maximum length (50 words)
* sampling caption for each image, while shuffling the image data
* encode captions into integer format based on look-up vocabulary index

In [ ]:
# sampling one caption per image
# return image_ids, caption_ids

def sampling_img_cap(data):
    
    # YOUR CODE HERE
    
    return image_ids, caption_ids


In [ ]:
# transform raw text caption into integer sequences of fixed maximum length

def prepare_caption(caption_ids, caption_data):
    
    # YOUR CODE HERE
    
    caption_seqs = 
    
      
    return caption_seqs

In [ ]:
# DO NOT CHANGE BELOW CODE

train_caps = []
for cap in train_data['caps']:
    train_caps.append(cap.decode())

val_caps = []
for cap in val_data['caps']:
    val_caps.append(cap.decode())

In [ ]:
# DO NOT CHANGE BELOW CODE

train_image_ids, train_caption_ids = sampling_img_cap(train_data)
val_image_ids, val_caption_ids = sampling_img_cap(val_data)

x_caption = prepare_caption(train_caption_ids, train_caps)
x_image = train_data['ims'][np.array(train_image_ids)]

x_val_caption = prepare_caption(val_caption_ids, val_caps)
x_val_image = val_data['ims'][np.array(val_image_ids)]

### 4. Create noise set for negative examples of image-fake caption and dummy output

Notice that we do not have real output with labels for training the model. Keras architecture expects labels, so we need to create dummy output -- which is numpy array of zeros. This dummy labels or output is never used since we compute loss function based on margin between positive examples (image-real caption) and negative examples (image-fake caption).

In [ ]:
# YOUR CODE HERE

train_noise = 
val_noise = 

y_train_labels = 
y_val_labels = 

### 5. Training model

In [ ]:
# YOUR CODE HERE

X_train = 
Y_train = 
X_valid = 
Y_valid = 

In [ ]:
# YOUR CODE HERE

# fit the model on training and validation set

#### Storing models and weight parameters

In [ ]:
# DO NOT CHANGE BELOW CODE

# Save model
training_model.save(os.path.join(MODEL_PATH,'image_caption_model.h5'))
# Save weight parameters
training_model.save_weights(os.path.join(MODEL_PATH, 'weights_image_caption.hdf5'))

# Save model for encoding caption and image
caption_model.save(os.path.join(MODEL_PATH,'caption_model.h5'))
image_model.save(os.path.join(MODEL_PATH,'image_model.h5'))

### 6. Feature extraction (Neural codes)

In [ ]:
# YOUR CODE HERE

# Use caption_model and image_model to produce "Neural codes" 
# for both image and caption from validation set

### 7. Caption Retrieval

#### Display original image as query and its ground truth caption

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing import image

In [ ]:
# YOUR CODE HERE

# choose one image_id from validation set
# use this id to get filepath of image
img_id = 
filepath_image = 

# display original caption
original_caption = 
print(original_caption)

# DO NOT CHANGE BELOW CODE
img = image.load_img(os.path.join(IMAGE_DATA,filepath_image), target_size=(224,224))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
# function to retrieve caption, given an image query

def get_caption(image_filename, n=10):   
    
    # YOUR CODE HERE


In [ ]:
# DO NOT CHANGE BELOW CODE
get_caption(filepath_image)

Briefly discuss the result. Why or how it works, and why do you think it does not work at some point.

#### Answer:

=== write your answer here ===

### 8. Image Retrieval

In [ ]:
# given text query, display retrieved image, similarity score, and its original caption 

def search_image(text_caption, n=10):
    
    # YOUR CODE HERE
    

Consider to use the following settings for image retrieval task.

* use real caption that is available in validation set as a query.
* use part of caption as query. For instance, instead of use the whole text sentence of the
caption, you may consider to use key phrase or combination of words that is included in
corresponding caption.

In [ ]:
# Example of text query 
# text = 'two giraffes standing near trees'

# YOUR QUERY-1
text1 = 

# DO NOT CHANGE BELOW CODE
search_image(text1)

In [ ]:
# YOUR QUERY-2
text2 = 

# DO NOT CHANGE BELOW CODE
search_image(text2)

Briefly discuss the result. Why or how it works, and why do you think it does not work at some point.

#### Answer:

=== write your answer here ===